In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

# ステートの定義
class State(TypedDict):
    # データを保存する属性
    messages: Annotated[list, add_messages]

# ステートグラフの作成
graph_builder = StateGraph(State)

In [4]:
# 言語モデルの定義
llm = ChatOpenAI(model_name=MODEL_NAME)

# チャットボットノードの作成
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# グラフにチャットボットノードを追加
graph_builder.add_node("chatbot", chatbot)

# 開始ノードの指定
graph_builder.set_entry_point("chatbot")
# 終了ノードの指定
graph_builder.set_finish_point("chatbot")

# 実行可能なステートグラフの作成
graph = graph_builder.compile()

In [5]:
# グラフの実行と結果の表示
def stream_graph_updates(user_input: str):
    # 結果をストリーミングで得る
    events = graph.stream({"messages": [("user", user_input)]})
    for event in events:
        for value in event.values():
            print("回答:", value["messages"][-1].content, flush=True)

# チャットボットのループ
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    print("質問:", user_input, flush=True)
    stream_graph_updates(user_input)

質問: 光の三原色
回答: 光の三原色は、赤（Red）、緑（Green）、青（Blue）の3つの色です。これらの色を組み合わせることで、さまざまな色を作り出すことができます。光の三原色は、加法混色の原理に基づいています。

具体的には、以下のようになります：

- 赤と緑を混ぜると、黄色になります。
- 赤と青を混ぜると、マゼンタ（紫）になります。
- 緑と青を混ぜると、シアンになります。
- すべての色を混ぜると、白色になります。

この原理は、テレビやコンピュータのディスプレイなど、光を使って色を表現する技術に広く利用されています。
ありがとうございました!


In [6]:
from langgraph.checkpoint.memory import MemorySaver

# チェックポインタの作成
memory = MemorySaver()

# 記憶を持つ実行可能なステートグラフの作成
memory_graph = graph_builder.compile(checkpointer=memory)

In [8]:
# グラフの実行と結果の表示
def stream_graph_updates(user_input: str):
    events = memory_graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    # 結果をストリーミングで得る
    for event in events:
        print(event["messages"][-1].content, flush=True)

# チャットボットのループ
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    stream_graph_updates(user_input)

こんにちわ
こんにちは！今日はどんなことをお話ししましょうか？
aで始まる単語を5つおしえて
もちろんです！以下は「a」で始まる単語を5つ挙げます：

1. apple（リンゴ）
2. airplane（飛行機）
3. artist（アーティスト）
4. animal（動物）
5. adventure（冒険）

他に知りたいことがあれば教えてください！
５つめの単語を別の単語にかえて
もちろんです！5つめの単語を別の単語に変えますね。

1. apple（リンゴ）
2. airplane（飛行機）
3. artist（アーティスト）
4. animal（動物）
5. activity（活動）

他に何か知りたいことがあれば教えてください！
４つめの単語の別の言い方をおしえて
4つめの単語「animal（動物）」の別の言い方として、「creature（生き物）」があります。他にも「beast（獣）」なども使われることがあります。何か他に知りたいことがあれば教えてください！
ありがとうございました!
